<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Методы машинного обучения</center></h1>
<h3><center> Шестаков Андрей (<a href="mailto:avshestakov@hse.ru">avshestakov@hse.ru</a>)</center></h3>
<hr>
<h2><center>Введение в рекомендательные системы</center></h2>


In [ ]:
%matplotlib inline

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

# Подготовка данных

1. Загрузите рейтинги (user_ratedmovies.dat) и описание фильмов (movies.dat)
2. С помощью LabelEncoder перекодируйте идентификаторы фильмов и юзеров в обоих датафреймах


In [ ]:
## Your Code Here

# Сжатое представление фильмов

1. С помощью from scipy.sparse.coo_matrix составьте разреженную матрицу рейтингов
2. С помощью scipy.sparse.linalg.svds получите латентное описание фильмов и пользователей
3. Для каждого фильма найдите 10 ближайших соседей в этих признаках по косинусной мере

In [ ]:
## Your Code Here

# User-based CF

In [ ]:
filepath = './data/user_ratedmovies_train.dat'
idx = df_rates.datetime < q
df_rates.loc[idx].to_csv(filepath, sep='\t', columns=['userID', 'movieID', 'rating'], index=None)

filepath = './data/user_ratedmovies_test.dat'
df_rates.loc[~idx].to_csv(filepath, sep='\t', columns=['userID', 'movieID', 'rating'], index=None)

In [ ]:
!pip install surprise

In [ ]:
from surprise import Dataset, Reader

In [ ]:
filepaths = [('./data/user_ratedmovies_train.dat', './data/user_ratedmovies_test.dat')]
reader = Reader(line_format='user item rating', sep='\t', skip_lines=1)
data = Dataset.load_from_folds(filepaths, reader=reader)

In [ ]:
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise.accuracy import rmse
from surprise import dump

Описание алгоритмов, основанных на CF - [туть](http://surprise.readthedocs.io/en/stable/knn_inspired.html)

In [ ]:
sim_options = {'name': 'pearson',
               'user_based': True,
               'min_support': 5,
               }

In [ ]:
dumpfile = './alg.dump'

In [ ]:
algo = KNNWithMeans(k=50, min_k=5, sim_options=sim_options)                                                       

for trainset, testset in data.folds(): 
    algo.train(trainset)                             
    predictions = algo.test(testset)
    rmse(predictions)
    
    dump.dump(dumpfile, predictions, algo)

In [ ]:
df_predictions = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details']) 

In [ ]:
df_predictions.head()

In [ ]:
algo.predict('1198', '5732')

In [ ]:
anti_train = trainset.build_anti_testset()

In [ ]:
user_id = '1897'
one_user = filter(lambda r: r[0] == user_id, anti_train)

In [ ]:
from collections import defaultdict

def get_top_n(predictions, n=10):

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
anti_train_predictions = algo.test(one_user, verbose=0)

In [ ]:
top_n = get_top_n(anti_train_predictions, n=10)

for uid, user_ratings in top_n.items():
    for (iid, _) in user_ratings:
        print(df_movies.loc[iid, 'title'])

In [ ]:
top_n